In [1]:
# !pip install tensorflow==1.15
# !pip install numpy==1.19

In [2]:
# from google.colab import drive

# drive.mount('/content/gdrive')

In [3]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [5]:
#!pip install tensorflow==1.15

In [6]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 256
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps,num_input])
Y = tf.placeholder("float", [None, num_classes])

In [7]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [8]:
def RNN(x, weights, biases):
  x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
  lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias = 1)

  outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
  return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [9]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,name = 'Adam')
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
# batch_x, batch_y = mnist.train.next_batch(batch_size)
# batch_x = batch_x.reshape((batch_size, timesteps, num_input))
# batch_x.shape
#  tf.transpose(batch_x,perm = [1,0,2])

In [11]:
# Start training
import time
  
# store starting time
begin = time.time()
  
loss_array = np.array([]);
accuracy_array = np.array([]);
with tf.Session() as sess:
  
    # Run the initializer
  sess.run(init)

  for step in range(1, training_steps+1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
  
        # Run optimization op (backprop)
    sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
    if step % display_step == 0 or step == 1:
      loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,Y: batch_y})
      loss_array = np.append(loss_array,loss)
      accuracy_array = np.append(accuracy_array,acc)
      print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
  time.sleep(1)
# store end time
  end = time.time()
  time_taken = end-begin
  print("Optimization Finished!")

  # Calculate accuracy for 128 mnist test images
  test_len = 128
  test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
  test_label = mnist.test.labels[:test_len]
  print("Testing Accuracy:",sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.6455, Training Accuracy= 0.094
Step 200, Minibatch Loss= 0.2095, Training Accuracy= 0.930
Step 400, Minibatch Loss= 0.1033, Training Accuracy= 0.957
Step 600, Minibatch Loss= 0.0883, Training Accuracy= 0.973
Step 800, Minibatch Loss= 0.0751, Training Accuracy= 0.965
Step 1000, Minibatch Loss= 0.0345, Training Accuracy= 0.988
Step 1200, Minibatch Loss= 0.0547, Training Accuracy= 0.977
Step 1400, Minibatch Loss= 0.0654, Training Accuracy= 0.984
Step 1600, Minibatch Loss= 0.0398, Training Accuracy= 0.992
Step 1800, Minibatch Loss= 0.0233, Training Accuracy= 0.992
Step 2000, Minibatch Loss= 0.0216, Training Accuracy= 0.996
Step 2200, Minibatch Loss= 0.0179, Training Accuracy= 0.996
Step 2400, Minibatch Loss= 0.0309, Training Accuracy= 0.996
Step 2600, Minibatch Loss= 0.0063, Training Accuracy= 1.000
Step 2800, Minibatch Loss= 0.0364, Training Accuracy= 0.988
Step 3000, Minibatch Loss= 0.0309, Training Accuracy= 0.996
Step 3200, Minibatch Loss= 0.0196, Training Acc

In [12]:
np.savetxt("tf.contrib.rnn.LSTMBlockCell_loss.csv", loss_array, delimiter=",")
np.savetxt("tf.contrib.rnn.LSTMBlockCell_acc.csv", accuracy_array, delimiter=",")

In [13]:
time_taken

71.71114349365234

In [15]:
np.savetxt("tf.contrib.rnn.LSTMBlockCell_time.csv", [time_taken], delimiter=",")